In [1]:
from langchain_community.llms import SparkLLM
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
import torch

参考文档
https://python.langchain.ac.cn/docs/integrations/llms/sparkllm/ \
推荐申请Spark Pro的免费token数

In [2]:
# 通过api调用大模型
llm = SparkLLM(
    spark_app_id="spark_app_id",
    spark_api_key="spark_api_key",
    spark_api_secret="spark_api_secret",
    spark_api_url="wss://spark-api.xf-yun.com/v3.1/chat",
    model="generalv3"
)

In [3]:
from langchain.document_loaders import PyPDFLoader

###加载文件
loader = PyPDFLoader("data/中国科学技术大学研究生学籍管理实施细则.pdf")
pages = loader.load()

from langchain.text_splitter import RecursiveCharacterTextSplitter

###文本切分
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 300,chunk_overlap = 50,)

docs = text_splitter.split_documents(pages)

预训练模型下载地址
https://hf-mirror.com/BAAI/bge-m3

In [ ]:
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_community.vectorstores import FAISS


embeddings = HuggingFaceBgeEmbeddings(
    model_name="bge-m3",
    model_kwargs = {'device': torch.device('cuda' if torch.cuda.is_available() else 'cpu')}
)

db = FAISS.from_documents(docs, embeddings)

In [5]:
augmented_prompt = """Using the contexts below, answer the query.

contexts:
{source_knowledge}

query: {query}"""

prompt = PromptTemplate(template=augmented_prompt, input_variables=["source_knowledge" ,"query"])
llm_chain = LLMChain(prompt=prompt, llm=llm  , llm_kwargs = {"temperature":0, "max_length":1024})

In [6]:
query = "问题"
result_simi = db.similarity_search(query , k=3)
source_knowledge = "\n".join([x.page_content for x in result_simi])
print(llm_chain.run( {"source_knowledge":source_knowledge ,"query" : query }))

**测试** \
测试在问答对.xlsx上的准确率